<a href="https://colab.research.google.com/github/lorenzopaoria/Clone-all-repositories-from-organization/blob/main/cigarettes_model_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms
from PIL import Image

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
def load_model(model_path):
    model = models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model

In [19]:
def get_predictions(model, image, threshold=0.5):
    transform = transforms.ToTensor()
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        predictions = model(image_tensor)

    boxes = predictions[0]['boxes']
    scores = predictions[0]['scores']
    labels = predictions[0]['labels']

    keep = scores > threshold
    boxes = boxes[keep]
    labels = labels[keep]
    return boxes, labels

In [20]:
def draw_boxes(image, boxes, labels):
    image = np.array(image)
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = box.tolist()
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    return image

In [21]:
def process_images(model_path, images_folder, output_folder):
    model = load_model(model_path)
    os.makedirs(output_folder, exist_ok=True)

    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)
        if image_path.endswith(('.jpg', '.png', '.jpeg')):
            print(f"Processing {image_name}")
            image = Image.open(image_path).convert("RGB")
            boxes, labels = get_predictions(model, image)

            result_image = draw_boxes(image, boxes, labels)

            result_image_path = os.path.join(output_folder, f"output_{image_name}")
            cv2.imwrite(result_image_path, result_image)
            print(f"Saved output to {result_image_path}")

In [22]:
def main():
    model_path = '/content/drive/MyDrive/pth_cigarette_detect/best_model.pth'
    images_folder = '/content/drive/MyDrive/Photo/test'
    output_folder = '/content/drive/MyDrive/test_trained'

    process_images(model_path, images_folder, output_folder)

if __name__ == "__main__":
    main()

<ipython-input-18-9d0b3382d6a5>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


RuntimeError: Error(s) in loading state_dict for FasterRCNN:
	size mismatch for roi_heads.box_predictor.cls_score.weight: copying a param with shape torch.Size([4, 1024]) from checkpoint, the shape in current model is torch.Size([91, 1024]).
	size mismatch for roi_heads.box_predictor.cls_score.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([91]).
	size mismatch for roi_heads.box_predictor.bbox_pred.weight: copying a param with shape torch.Size([16, 1024]) from checkpoint, the shape in current model is torch.Size([364, 1024]).
	size mismatch for roi_heads.box_predictor.bbox_pred.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([364]).